# Emerging Technologies Tasks

My solutions to the Emerging Technologies tasks assessment.

## Contents

- [Task 1: Square Root of 2](#Task-1:-Square-Root-of-2)

## Task 1: Square Root of 2

>Write a Python function called `sqrt2` that calculates and prints to the screen the square root of 2 to 100 decimal places.
>Your code should not depend on any module from the standard library or otherwise. You should research the task first and include references and a description of your algorithm.

### Algorithm Description

Before discussing the algorithm used to solve this problem, it is neccessary to first provide some backgound on floating point numbers. 

#### Floating Point Numbers

Floating point numbers are used to represent fractional values and are stored in a computer as base 2 (binary) fractions. Most decimal fractions cannot be accurately represented as binary fractions and as a result, decimal floating-point numbers are only approximated by the binary floating-point numbers actually stored in the machine [1]. For instance, the summation of $0.1$ and $0.2$ yields a result of $0.3$ in decimal. However, the result of this computation cannot be accurately represented in binary (see below).

In [1]:
0.1 + 0.2

0.30000000000000004

The IEEE standard defines 32-bit and 64-bit floating-point representations. The 32-bit (single-precision) format consists of a sign bit, an 8-bit exponent and 23 bits of mantissa. The 64-bit (double-precision) format consists of a sign bit, an 11-bit exponent and 52 bits of mantissa. Floating-point arithmetic on computers is therefore inherently inexact as the 24 bits of mantissa in a 32-bit floating-point number can only represent approximately 7 significant decimal digits (for doubles this is 15-17 decimal digits [3]). If a number is not exactly representable, then it must instead be approximated [2].

Python's float type has double-precision [1], and it is therefore not possible to store a number with 100 decimal places as a float. However, in Python integers have unlimited precision [4], so one approach to the given problem is to store all the significant digits of $\sqrt2$ as an integer. Finding the square root of a given number $p$, as an integer can be achieved by calculating the *integer square root* of $p*10^{2 * q}$, where $q$ is the number of digits after the decimal point, in this case 100.

#### Finding the Integer Square Root using Newton's Method

The integer square root of a number $n$ is the greatest positive integer less than or equal to the square root of $n$ [5], and can be found using Newton's method. Newton's algorithm works by producing successively better approximations of the roots (or zeroes) of a real-valued function [6]. It starts with some guess $z$, and then adjusts $z$ based on how close $z^{2}$ is to $x$, producing a better guess [7]. The process is continuously repeated until a sufficiently accurate value is reached.

### Solution

Below is my solution to task one. I start by defining a function, `isqrt` that calculates the integer square root of a number. I then define the `sqrt2` function which uses `isqrt` to calculate the square root of 2 to 100 decimal places and prints the result to the screen.

In [2]:
def isqrt(n: int) -> int:
    """
    Calculates the integer square root of `n` using Newton's method.
    :return: the integer square root of `n`.
    """
    # Reference: user448810 - https://stackoverflow.com/a/15391420
    x = n

    # Calculate the initial guess value
    z = (x + 1) // 2

    while z < x:
        x = z
        z = (x + n // x) // 2

    return x

In [3]:
def sqrt2():
    """
    Calculates and prints to the screen the square root of 2 to 100 decimal places.
    """
    # Set the number of places to show after the decimal point
    precision = 100

    # Get the integer square root & multiply by the required power of 10
    # Reference: Eugene Yarmash - https://stackoverflow.com/a/32651586
    result = str(isqrt(2 * 10 ** (2 * precision))) # 2 x 10^200

    ## Insert the decimal point
    result = result[:1] + '.' + result[1:]

    ## Output the result
    print(result)

### Testing

In order to help illustrate the functions accuracy, we can compare the output of the standard library's `math.sqrt()` function with the above `sqrt2()` function.

In [4]:
from math import sqrt

print(sqrt(2))
sqrt2()

1.4142135623730951
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727


We can also compare the result to that given by the `sqrt` function provided by Pyhon's built-in `decimal` module. Doing so, we find both functions produce the same number.

In [5]:
from decimal import (
    getcontext,
    Decimal
)

getcontext().prec = 101
result = Decimal(2).sqrt()

print(result)
sqrt2()

1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727


## References

1. [*Floating Point Arithmetic: Issues and Limitations*](https://docs.python.org/3/tutorial/floatingpoint.html). The Python 3 Standard Library Documentation.
2. [*The Perils of Floating Point*](http://www.lahey.com/float.htm). Bush, Bruce M.
3. [*Double-precision floating-point format*](https://en.wikipedia.org/wiki/Double-precision_floating-point_format). Wikipedia.
4. [*Built-in Types*](https://docs.python.org/3/library/stdtypes.html). The Python 3 Standard Library Documentation.
5. [*Integer square root*](https://en.wikipedia.org/wiki/Integer_square_root). Wikipedia.
6. [*Newton's Method*](https://en.wikipedia.org/wiki/Newton%27s_method). Wikipedia.
7. [*A Tour of Go*](https://tour.golang.org/flowcontrol/8). Golang documentation.